In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from pprint import pformat

from hloc import extract_features, match_features, pairs_from_retrieval, pairs_from_ios_poses
from hloc import reconstruction, localize_sfm, visualization

from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d

# Pipeline for ANA image data

## Setup
Here we declare the paths to the dataset, the reconstruction and localization outputs, and we choose the feature extractor and the matcher. 

In [ ]:
dataset = Path('datasets/office2/')
images = Path('datasets/office2/frames/')
outputs = Path('outputs/office2/')

sfm_pairs = outputs / 'pairs-db-covis20.txt'  # top 20 most covisible in SIFT model
loc_pairs = outputs / 'pairs-query-netvlad20.txt'  # top 20 retrieved by NetVLAD
reference_sfm = outputs / 'sfm_superpoint+superglue'  # the SfM model we will build
sfm_dir = outputs / 'sfm'

# list the standard configurations available
# print(f'Configs for feature extractors:\n{pformat(extract_features.confs)}')
# print(f'Configs for feature matchers:\n{pformat(match_features.confs)}')

In [ ]:
# pick one of the configurations for image retrieval, local feature extraction, and matching
# you can also simply write your own here!
retrieval_conf = extract_features.confs['netvlad']
feature_conf = extract_features.confs['disk']
matcher_conf = match_features.confs['disk+lightglue']

## Extract local features for database and query images

In [ ]:
features = extract_features.main(feature_conf, images, outputs)

The function returns the path of the file in which all the extracted features are stored.

In [ ]:
# This is the file where the features were saved
print(f'Features were exported to file: {features}')

# How can we plot the exported features?
from hloc.utils.io import list_h5_names, read_image, get_keypoints
from hloc.utils.viz import plot_images, plot_keypoints

sample_image_names_list = list_h5_names(features)

sample_idx = 3
sample_image_name = sample_image_names_list[sample_idx] #'db/1931.jpg'
sample_image_path = images / sample_image_name
sample_image = read_image(sample_image_path)
sample_image_kps = get_keypoints(features, sample_image_name)

plot_images([sample_image])
plot_keypoints([sample_image_kps])


## Generate pairs for the SfM reconstruction
Instead of matching all database images exhaustively, we exploit the existing SIFT model to find which image pairs are the most covisible. We first convert the SIFT model from the NVM to the COLMAP format, and then do a covisiblity search, selecting the top 20 most covisibile neighbors for each image.

In [ ]:
# Define the file paths
poses_file_path = Path.home() / 'Anantak/Pipelines/Hierarchical-Localization/datasets/office2/poses.txt'
rotation_threshold = 15

# Call the main function of your script
pairs_from_ios_poses.main(
    poses_file=poses_file_path,
    output=sfm_pairs,
    num_matched=8000,
    rotation_threshold=rotation_threshold
)

## Match the database images

In [ ]:
# Call the main function of match_features
sfm_matches = match_features.main(matcher_conf, sfm_pairs, feature_conf['output'], outputs)
print(f'sfm matches were saved in file {sfm_matches}')

The function returns the path of the file in which all the computed matches are stored.

## Triangulate a new SfM model from the given poses
We triangulate the sparse 3D pointcloud given the matches and the reference poses stored in the SIFT COLMAP model.

In [ ]:
sfm_dir = Path('/home/ubuntu/Anantak/Pipelines/Hierarchical-Localization/datasets/office2/frames/')
images_path = Path('/home/ubuntu/Anantak/Pipelines/Hierarchical-Localization/datasets/office2/frames/')
sfm_pairs_path = Path('/home/ubuntu/Anantak/Pipelines/Hierarchical-Localization/outputs/office2/pairs-db-covis20.txt')
features_path = Path('/home/ubuntu/Anantak/Pipelines/Hierarchical-Localization/outputs/office2/feats-disk.h5')
matches_path = Path('/home/ubuntu/Anantak/Pipelines/Hierarchical-Localization/outputs/office2/feats-disk_matches-disk-lightglue_pairs-db-covis20.h5')

model = reconstruction.main(
    sfm_dir=sfm_dir,
    image_dir=images_path,
    pairs=sfm_pairs_path,
    features=features_path,
    matches=matches_path
    )

In [ ]:
fig = viz_3d.init_figure()
viz_3d.plot_reconstruction(fig, model, color='rgba(255,0,0,0.5)', name="mapping", points_rgb=True)
fig.show()

## Find image pairs via image retrieval
We extract global descriptors with NetVLAD and find for each image the $k$ most similar ones. A larger $k$ improves the robustness of the localization for difficult queries but makes the matching more expensive. Using $k{=}10{-}20$ is generally a good tradeoff but $k{=}50$ gives the best results for the Aachen Day-Night dataset.

In [ ]:
global_descriptors = extract_features.main(retrieval_conf, images, outputs)
pairs_from_retrieval.main(global_descriptors, loc_pairs, num_matched=20, db_prefix="", query_prefix="")

## Match the query images

In [ ]:
loc_matches = match_features.main(matcher_conf, loc_pairs, feature_conf['output'], outputs)

## Localize!
Perform hierarchical localization using the precomputed retrieval and matches. The file `Aachen_hloc_superpoint+superglue_netvlad50.txt` will contain the estimated query poses. Have a look at `Aachen_hloc_superpoint+superglue_netvlad50.txt_logs.pkl` to analyze some statistics and find failure cases.

In [ ]:
# results = outputs / 'Aachen_hloc_superpoint+superglue_netvlad20.txt'  # the result file
# localize_sfm.main(
#     reconstruction,
#     dataset / 'queries/*_time_queries_with_intrinsics.txt',
#     loc_pairs,
#     features,
#     loc_matches,
#     results,
#     covisibility_clustering=False)  # not required with SuperPoint+SuperGlue

reference_sfm = Path('datasets/office2/frames')
queries = Path('datasets/office2/frames/query_with_intrinsics.txt')
features = Path('outputs/office2/feats-disk.h5')
matches = Path('outputs/office2/feats-disk_matches-disk-lightglue_pairs-query-netvlad20.h5')
retrieval = Path('outputs/office2/pairs-query-netvlad20.txt')
results = Path('outputs/office2/estimated_poses.txt')

localize_sfm.main(
    reference_sfm=reference_sfm,
    queries=queries,
    retrieval=retrieval,
    features=features,
    matches=matches,
    results=results,
    ransac_thresh=12.0,
    covisibility_clustering=False,
    prepend_camera_name=False
)

## Visualizing the SfM model
We visualize some of the database images with their detected keypoints.
Color the keypoints by track length: red keypoints are observed many times, blue keypoints few.

In [ ]:
visualization.visualize_sfm_2d(reconstruction, images, n=1, color_by='track_length')

Color the keypoints by visibility: blue if sucessfully triangulated, red if never matched.

In [ ]:
visualization.visualize_sfm_2d(reconstruction, images, n=1, color_by='visibility')

Color the keypoints by triangulated depth: red keypoints are far away, blue keypoints are closer.

In [ ]:
visualization.visualize_sfm_2d(reconstruction, images, n=1, color_by='depth')

## Visualizing the localization
We parse the localization logs and for each query image plot matches and inliers with a few database images.

In [ ]:
visualization.visualize_loc(
    results, images, reconstruction, n=1, top_k_db=1, prefix='', seed=2)